In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from utils import *
from imblearn.under_sampling import RandomUnderSampler
from sklearn.decomposition import PCA
from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, IterativeImputer, BiScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import chi2
import warnings
from sklearn.linear_model import LassoCV
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [3]:
##### load & clean data
train, test = load_data()
train = train.dropna(axis=0, how='all') # remove missing rows
y_train = train.Class
train = train.drop(columns='Class')

#columns_to_drop = ['Pre_L_Pupil_Day', 'T_L_Actual_AblDepth']
#train, test = remove_columns(train, test, columns_to_drop)

print("train.shape without missing rows =", train.shape)
print("class count:\n", y_train.value_counts())
##### add here new steps before predict (TODO: undersample)
##### add here new steps to preprocess
preprocessing = Pipeline([
                ('scaler', StandardScaler()),
                ('knn-imputation', KNN(5))
                ])
X_train, X_test = preprocess_data(train, test, preprocessing)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, stratify=y_train, random_state=42)
evaluation =  Pipeline([
#                 ('smote', SMOTE(random_state=42, ratio=1, n_jobs=-1, k_neighbors=3)),
#                 ('random_oversample', RandomOverSampler()),
                ('xg', XGBClassifier(n_jobs=-1, n_estimators=40, tree_method='gpu_exact', predictor='gpu_predictor', random_state=52, scale_pos_weight=1.2,max_depth=4))
               # ('select_from_model',SelectFromModel(selection_model, threshold=0.071, prefit=True))
                ])
# evaluation.get_params().keys()
gs = evaluation
gs.fit(X_train, y_train)  
# print('best params:\n\t{}'.format(gs.best_params_))
# print('best auc score:\n\t{:.4f}'.format(gs.best_score_))

x_val_pred = gs.predict_proba(X_val)[:,1]
print('auc score on x_val:\n\t{:.4f}'.format(roc_auc_score(y_val, x_val_pred)))
pred = gs.predict_proba(X_test)[:,1]
to_file(pred, "tal_test_k5")

train.shape without missing rows = (30451, 51)
class count:
 0.0    29949
1.0      502
Name: Class, dtype: int64
Imputing row 1/42903 with 10 missing, elapsed time: 785.651
Imputing row 101/42903 with 10 missing, elapsed time: 785.938
Imputing row 201/42903 with 9 missing, elapsed time: 786.154
Imputing row 301/42903 with 21 missing, elapsed time: 786.370
Imputing row 401/42903 with 21 missing, elapsed time: 786.602
Imputing row 501/42903 with 8 missing, elapsed time: 786.824
Imputing row 601/42903 with 13 missing, elapsed time: 787.041
Imputing row 701/42903 with 10 missing, elapsed time: 787.273
Imputing row 801/42903 with 10 missing, elapsed time: 787.689
Imputing row 901/42903 with 9 missing, elapsed time: 788.074
Imputing row 1001/42903 with 21 missing, elapsed time: 788.444
Imputing row 1101/42903 with 10 missing, elapsed time: 788.670
Imputing row 1201/42903 with 15 missing, elapsed time: 788.913
Imputing row 1301/42903 with 12 missing, elapsed time: 789.129
Imputing row 1401/42